In [22]:
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch
from torch import nn
import torch
from torch.utils.data import DataLoader, Subset
import os


In [ ]:


class WeightAligning_NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_old_classes):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_old_classes = num_old_classes
        
        # Definir las capas lineales
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Propagación hacia adelante en la red
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        if self.output_size != self.num_old_classes: 
            # Separar los pesos de las capas lineales para clases antiguas y nuevas
            weights_old = self.fc3.weight[:self.num_old_classes, :]
            weights_new = self.fc3.weight[self.num_old_classes:, :]
            

            # Calcular las normas de los vectores de peso para clases antiguas y nuevas
            norm_old = torch.norm(weights_old, dim=1)
            norm_new = torch.norm(weights_new, dim=1)
            
            # Calcular el factor de normalización γ
            gamma = torch.mean(norm_old) / torch.mean(norm_new)
            
            # Aplicar el alineamiento de pesos (Weight Aligning)
            weights_new_aligned = gamma * weights_new
            
            # Concatenar los pesos alineados con los pesos antiguos
            weights_aligned = torch.cat((weights_old, weights_new_aligned), dim=0)
            
            # Aplicar los pesos alineados para calcular la salida
            logits = F.linear(x, weights_aligned, self.fc3.bias)

        else: logits = self.fc3(x)
        
        return logits


In [ ]:

import torch
import torch.nn as nn

# Definir las dimensiones de la red y el número de clases antiguas
input_size = 28*28  # Tamaño de entrada (por ejemplo, para imágenes de 28x28 píxeles)
hidden_size = 512  # Tamaño de las capas ocultas
output_size = 10  # Tamaño de salida (ejemplo: 20 clases nuevas)
num_old_classes = 5  # Número de clases antiguas

# Crear una instancia de la red neuronal con alineación de pesos
model = WeightAligning_NeuralNetwork(input_size, hidden_size, output_size, num_old_classes)

# Crear datos de entrada de ejemplo
input_data = torch.randn(1, input_size)  # Supongamos una sola imagen de tamaño input_size

# Pasar los datos de entrada a través de la red neuronal
output_logits = model(input_data)

# Mostrar la forma de los logits de salida
print("Forma de los logits de salida:", output_logits.shape)
